## Day 10: Knot Hash

http://adventofcode.com/2017/day/10

### Part 1

Circular lists again, it's probably time to define some utility functions. And it's probably time to stop using mutable data structures (unless they're really convenient).

In [1]:
from itertools import cycle, islice
from pyrsistent import pvector 

def cget(v, i):
    return v[i % len(v)]

def cset(v, i, val):
    return v.set(i % len(v), val)

def cslice(v, start, stop):
    # This starts getting slow for large numbers,
    # so minimise the number of cycles produced.
    length = stop - start
    start = start % len(v)
    return islice(cycle(v), start, start + length)

In [2]:
def knot_step(string, length, current):
    r = reversed(pvector(cslice(string, current, current + length)))
    result = pvector(string)
    for i, c in zip(range(current, current + length), r):
        result = cset(result, i, c)
    return result

def knot(string, lengths):
    current = 0
    skip = 0
    
    for length in lengths:
        string = knot_step(string, length, current)
        current = current + length + skip
        skip += 1
        
    return string

In [3]:
test_lengths = [3, 4, 1, 5, 0]
test_string = pvector(range(5))

knot(test_string, test_lengths)

pvector([3, 4, 2, 1, 0])

Jolly good.

In [4]:
input_lengths = (int(x) for x in open('input', 'r').read().split(','))
input_string = pvector(range(256))

x = knot(input_string, input_lengths)
x[0] * x[1]

15990

### Part 2

This gives the wrong answer. I might come back to it.

First convert to ascii and add the extra specified.

In [5]:
def ascii_codes(xs):
    return [ord(x) for x in xs]

def length_sequence(s):
    return ascii_codes(s) + [17, 31, 73, 47, 23]

assert length_sequence('1,2,3') == [49,44,50,44,51,17,31,73,47,23]

Amend the knot function so that it takes a number of rounds.

In [6]:
def knot(string, lengths, rounds=1):
    current = 0
    skip = 0
    
    for i in range(rounds):
        for length in lengths:
            string = knot_step(string, length, current)
            current = current + length + skip
            skip += 1
        
    return string

Splits a sequence into equal-sized lengths.

In [7]:
def split_by_length(xs, length):
    for i in range(0, len(xs), length):
        yield xs[i:i+length]
        
list(split_by_length(pvector(range(256)), 16))

[pvector([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]),
 pvector([16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]),
 pvector([32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]),
 pvector([48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63]),
 pvector([64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79]),
 pvector([80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95]),
 pvector([96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111]),
 pvector([112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127]),
 pvector([128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143]),
 pvector([144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159]),
 pvector([160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175]),
 pvector([176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191]),
 pvect

Bitwise XOR each subsequence.

In [8]:
from functools import reduce
import operator

def dense_hash(sparse_hash):
    for xs in split_by_length(sparse_hash, 16):
        yield reduce(operator.xor, xs)

It took me quite a long time to realise I needed to zero-pad the hexadecimals.

In [9]:
def knot_hash(xs):
    return ''.join((['%02x' % x for x in dense_hash(knot(pvector(range(256)), length_sequence(xs), 64))]))

In [10]:
with open('input', 'r') as f:
    input_lengths = f.read().strip()

In [11]:
assert knot_hash('') == 'a2582a3a0e66e6e86e3812dcb672a272'

In [12]:
knot_hash(input_lengths)

'90adb097dd55dea8305c900372258ac6'

At last. 